# BTDP Next GenAI Services Notebook

## 🎯 Objectives

The objective of this notebook is to have a first hand on the Next GenAI Services.

You will learn how to use our `context` endpoint to see the information about your business context.

## ⚙️ Setup

- Go to your terminal 
- Run the script `00-init.sh`
- Inside this notebook select the `genaiservices_kernel` kernel

- Then identify yourself by executing the following command `! gcloud auth login`


# Imagen 3 Image Editing

<table align="left">

  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Foa-datafactory-services%2Fbtdp-samples%2Fmaster%2Fbtdp-platform-services%2Fgenai-service%2F01-context-management.ipynb?token=GHSAT0AAAAAAC2C2LHLWVIFH73E3VUE57GY2CBUKHQ">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
</table>

### 📖 Importing the required libraries

In [ ]:
import requests
import os
import json
import pandas as pd
from IPython import get_ipython

### ✏️ Setup the variables

In [ ]:
# Set the APIGEE base url + contexts endpoint
genaiconfig_base = "https://api.loreal.net/global/it4it/btdp-genaiconfig/v1"
contexts_endpoint = f"{genaiconfig_base}/contexts"

# Put your context_id provided by the GenAI Team!
context_id = "<YOUR_CONTEXT_ID_HERE>"

# Get your access token
identity_token = get_ipython().getoutput('gcloud auth print-access-token')[0]

# Put your access token as a header
headers = {
    "Authorization": f"Bearer {identity_token}"
}

### 📋 List the context you have access to

In [ ]:
response = requests.get(contexts_endpoint, headers=headers)
data = response.json()

df = pd.DataFrame(data)

print("Context you are able to see")
df


### 🔎 See the Access control of a context

In [ ]:
url = f"{contexts_endpoint}/{context_id}"

response = requests.get(url, headers=headers)

data = response.json()
print("Admins")
display(data["access_control"]["admins"])
print("------")
print("Users")
display(data["access_control"]["users"])
print("------")
print("OAPass")
display(data["oapass"])


## 👀 See your context details

In [ ]:
url = f"{contexts_endpoint}/{context_id}"

response = requests.get(url, headers=headers)

data = response.json()

print("Name")
display(data["name"])
print("------")
print("Short_desc")
display(data["short_desc"])
print("------")
print("Long_desc")
display(data["long_desc"])
print("------")
print("Links")
display(data["links"])
print("------")
print("Disclaimer")
display(data["disclaimer"])
print("------")
print("Contact")
display(data["contact"])

## ✍️ Edit your context

🚨 Be aware that you are overwriting all these fields. So be careful before running the next cell

As a context admin you are able to modify some fields:

- name
- short_desc
- long_desc
- links
    - how_to_use
    - feedbacks
- disclaimer
    - en
    - fr
- contact (list of max 2 mails)

In [ ]:
to_update = {
    "name": "Your companion name", # <-- Be careful every user of your companion can see that
    "short_desc": "Your short description", # <-- Be careful every user of your companion can see that
    "long_desc": "Your long description", # <-- Be careful every user of your companion can see that
    "links": {
        "how_to_use": "https://...", # <-- Optional: You can put the link you want (displayed in the details, below the long description)
        "feedbacks": "https://..."
    },
    "disclaimer": {
        "en": "Disclaimer in english", # <-- Optional: Disclaimer, showed below the prompt bar in L'orealGPT
        "fr": "Disclaimer in french"
    },
    "contact": [    # <-- Optional: The contact, if you don't want to put one just delete these 3 lines
        "my_email@loreal.com",
        "my_colleague_mail@loreal.com"
    ]
}

response = requests.put(
    f"{contexts_endpoint}/{context_id}/details",
    headers=headers,
    json=to_update
)

data = response.json()
print("New name")
display(data["name"])
print("------")
print("New short_desc")
display(data["short_desc"])
print("------")
print("New long_desc")
display(data["long_desc"])
print("------")
print("New links")
display(data["links"])
print("------")
print("New disclaimer")
display(data["disclaimer"])
print("------")
print("New contact")
display(data["contact"])



### Update the Access user or group based

In [ ]:
## BE CARREFULL
## It's not happend mode ! It overwrites the access so you have to put all the previous members into the group !
data = {
"access_control":
{'admins': [
    # It can be one or many users
    #{"name":"thomas.menard@loreal.com", "principal_type":"user"},

    # It can be also one or many groups
    #{"name":"antoine.castex@loreal.com", "principal_type":"group"},
            ],
 'users': [
    # It can be one or many users
    #{"name":"thomas.menard@loreal.com", "principal_type":"user"},

    # It can be also one or many groups
    #{"name":"antoine.castex@loreal.com", "principal_type":"group"},
  ]}


}
response = requests.patch(
    f"{contexts_endpoint}/{context_id}",
    headers=headers,
    json=data
)

data = response.json()
print("New Admins")
display(data["access_control"]["admins"])
print("------")
print("New Users")
display(data["access_control"]["users"])
print("------")
print("OAPass")
display(data["oapass"])

### Update the Access using OAPASS
- It's possible to use `Country` / `Entity` / `Contract Type` / `Job_Function_level1` to grant the access to a companion

In [ ]:
#Example of the kind of rule you can apply :
#  job_function_level1 in ("Campus","Corporate Responsibility","EHS & Facilities")
#  or entity in ("R&I N.A.","R&I RAD","R&I USA","R&I China")
#  or country in ("France") and contract_type in ("LongTerm Contract")

data = {
    "oapass":"country in (\"Australia\",\"New Zealand\")",
}

response = requests.patch(
    f"{contexts_endpoint}/{context_id}",
    headers=headers,
    json=data
)

data = response.json()
print("New Admins")
display(data["access_control"]["admins"])
print("------")
print("New Users")
display(data["access_control"]["users"])
print("------")
print("OAPass")
display(data["oapass"])

### Updating the name of the companion and the descriptions

In [ ]:
data = {
    "name":"-IT- Market research brief coach",
    # Short desc is 100 chars max"
    "short_desc":"Guides the marketer write a sharp and effective market research brief from scratch.",
    # Long desc is 1000 chars max"
    "long_desc":"""The market research brief coach guides the marketer write a sharp and effective market research brief from scratch.
"""
}

response = requests.patch(
    f"{contexts_endpoint}/{context_id}",
    headers=headers,
    json=data
)

data = response.json()
print("New name")
display(data["name"])
print("------")
print("New short_desc")
display(data["short_desc"])
print("------")
print("New long_desc")
display(data["long_desc"])